<a href="https://colab.research.google.com/github/Harbiodun0122/OCR-for-Nigerian-Licence-Plate-/blob/master/LicensePlateOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Install ultralytics and fast-plate-ocr
!pip install ultralytics fast-plate-ocr[onnx]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


In [3]:
# Import libraries
import cv2, os, re, csv
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from fast_plate_ocr import LicensePlateRecognizer

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Fast plate OCR

In [4]:
# Load the fast plate ocr model
plate_recognizer = LicensePlateRecognizer('cct-xs-v1-global-model')

In [5]:
# Nigerian plate patterns
NIGERIAN_PLATE_PATTERNS = [
    r'^[A-Z]{3}\d{3}[A-Z]{2}$',      # AAA123AA
    r'^[A-Z]{2}\d{3}[A-Z]{3}$',      # AA123AAA
    r'^[A-Z]{2}\d{2}[A-Z]\d{2}$',    # AA12A34
    r'^\d{2}[A-Z]\d{2}[A-Z]{2}$',    # 12A34AA
    r'^[A-Z]{4}\d{3}$',              # AAAA123
]

def format_plate_pattern(text):
  """
  Fornat the plate number to the correct pattern.
  """

  cleaned = text[0].upper().replace(' ', '').replace('_', '')

  # AAA123AA -> AAA-123AA
  if re.match(NIGERIAN_PLATE_PATTERNS[0], cleaned):
    return f"{cleaned[:3]}-{cleaned[3:]}"

  # AA123AAA -> AA123-AAA
  elif re.match(NIGERIAN_PLATE_PATTERNS[1], cleaned):
    return f"{cleaned[:-3]}-{cleaned[-3:]}"

  # AA12A34 -> AA12-A34
  elif re.match(NIGERIAN_PLATE_PATTERNS[2], cleaned):
    return f"{cleaned[:4]}-{cleaned[4:]}"

  # 12A34AA -> 12A-34AA
  elif re.match(NIGERIAN_PLATE_PATTERNS[3], cleaned):
    return f"{cleaned[:3]}-{cleaned[3:]}"

  # AAAA123 -> AAAA-123
  elif re.match(NIGERIAN_PLATE_PATTERNS[4], cleaned):
    return f"{cleaned[:4]}-{cleaned[4:]}"

  # If pattern do not match, return text as it is
  else:
    return cleaned

### Run Inference on a video

In [6]:
# Setting the paths
BASE_DIR = "/content/drive/MyDrive/Licence Plate OCR/"
video_path = os.path.join(BASE_DIR, "traffic.mp4")
csv_path = os.path.join(BASE_DIR, "detected_plates.csv")
model_path = os.path.join(BASE_DIR, "licence_detection_output/train/weights/best.pt")

**Record plate numbers in a CSV file**

In [7]:
# Load already-saved plates if file exists
saved_plates = set()

if os.path.exists(csv_path):
    with open(csv_path, newline="", mode="r") as f:
        reader = csv.reader(f)
        next(reader, None)  # skip header
        for row in reader:
            if row:
                saved_plates.add(row[0])

# Open CSV in append mode
csv_file = open(csv_path, mode="a", newline="")
csv_writer = csv.writer(csv_file)

# Write header only once
if csv_file.tell() == 0:
    csv_writer.writerow(["plate_number"])

In [8]:
from collections import defaultdict, Counter

# Per-track OCR buffers
ocr_buffer = defaultdict(list)

# How many frames before we trust OCR
MIN_OCR_FRAMES = 3

In [9]:
def fully_in_frame(x1, y1, x2, y2, weight, height, margin=5):
    return (
        x1 > margin and
        y1 > margin and
        x2 < weight - margin and
        y2 < height - margin
    )

In [10]:
model = YOLO(model_path)

# Open the video file and get video details
videoCap = cv2.VideoCapture(video_path)
width = int(videoCap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(videoCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_per_second = int(videoCap.get(cv2.CAP_PROP_FPS))
num_frames = int(videoCap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(os.path.join(BASE_DIR, "License Plate OCR.mp4"), fourcc, frame_per_second, (width, height))

# Loop through the video frames
while videoCap.isOpened():
  # Read a frame from the video
  success, frame = videoCap.read()

  if success:
    # Run YoloV11 tracking on the frames
    results = model.track(
    frame,
    persist=True,
    tracker="botsort.yaml",
    conf=0.4,
    iou=0.6
    )

    if results[0].boxes.id is not None:
        boxes = results[0].boxes
        print("boxes:", boxes)

        for box, track_id in zip(boxes, boxes.id):
            track_id = int(track_id)
            print("track_id: ", track_id)
            print("confidence:", box.conf[0])

            if box.conf[0] < 0.4:
                continue

            # Get the coordinates
            [x1, y1, x2, y2] = box.xyxy[0]

            # Convert to ints
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw rectangle on detected plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

            # Check if detected plate is fully in frame
            if not fully_in_frame(x1, y1, x2, y2, width, height):
                continue

            # Crop detected plate
            plate_img = frame[y1:y2, x1:x2]

            # Run OCR
            plate = plate_recognizer.run(plate_img)

            # Properly format plate number
            formatted_plate = format_plate_pattern(plate)
            print("formatted plate: ", formatted_plate)

            if formatted_plate:
                ocr_buffer[track_id].append(formatted_plate)

            buffer = ocr_buffer[track_id]
            print('buffer: ', buffer)

            if len(buffer) >= MIN_OCR_FRAMES:
                most_common, count = Counter(buffer).most_common(1)[0]
                print("Most common: ", most_common, "\nCount: ", count)

                if most_common and most_common not in saved_plates:
                    saved_plates.add(most_common)

                    # Save to CSV (append-only)
                    csv_writer.writerow([most_common])
                    csv_file.flush()

                # Draw stabilized label
                text_size, _ = cv2.getTextSize(most_common, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                text_w, text_h = text_size

                cv2.rectangle(
                    frame,
                    (x1, y1 - text_h - 10),
                    (x1 + text_w + 10, y1),
                    (255, 0, 0),
                    -1
                )

                cv2.putText(
                    frame,
                    most_common,
                    (x1 + 5, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (255, 255, 255),
                    2
                )

    out.write(frame)

  else:
    # Break the loop if the end of the video is reached
    break

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 130ms
Prepared 1 package in 41ms
Installed 1 package in 1ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 (no detections), 526.1ms
Speed: 18.6ms preprocess, 526.1ms inference, 20.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 343.6ms
Speed: 4.6ms preprocess, 343.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 326.6ms
Speed: 3.5ms preprocess, 326.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 319.3ms
Speed: 4.0ms preprocess, 319.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 343.2ms
Speed: 3.9ms preprocess, 343.2ms inference, 0.6ms postprocess pe

In [11]:
# Release the video capture object and close the display window
videoCap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()